# Introduction to rna-to-dna converter
It converts the rna structure generated by https://rnacomposer.cs.put.poznan.pl/ to a dna structure.  Run the first four boxes of the script below in order to generate an output file ...

In [22]:
import numpy as np
import sys
sys.path.append('/usr/local/lib/python3.7/site-packages/')
import MDAnalysis

In [23]:
input_file='n1.pdb'
output_file='n1_output.pdb'

In [24]:
u=MDAnalysis.Universe(f'{input_file}')
u.atoms.write('temp.gro',reindex=False)
u2 = MDAnalysis.Universe('temp.gro')

In [25]:
fmt="{resid:>5d}{resname:<5.5s}{name:>5.5s}{index:>5d}{pos[0]:8.3f}{pos[1]:8.3f}{pos[2]:8.3f}"

with open ('temp_final.gro', 'w') as f:
  i=1
  for res_id in np.unique(u2.atoms.resids):
    tmp=u2.select_atoms('resid '+str(res_id))
    if tmp.resnames[0]=='U':
        for x in range(len(tmp)):
            if tmp.atoms.names[x]!="O2'" and tmp.atoms.names[x]!="H2'" and tmp.atoms.names[x]!="H5":
               print(fmt.format(     resid=tmp.resids[0],
                                resname="DT",
                                index=i,
                                name=tmp.atoms.names[x],
                                pos=tmp.positions[x]/10),file=f)
               i+=1

            elif tmp.atoms.names[x]=="O2'":
                print(fmt.format(     resid=tmp.resids[0],
                                resname="DT",
                                index=i,
                                name=tmp.atoms.names[x+1],
                                pos=tmp.positions[x]/10),file=f)
                i+=1

            elif tmp.atoms.names[x]=="H5":
                print(fmt.format(     resid=tmp.resids[0],
                                resname="DT",
                                index=i,
                                name="C5M",
                                pos=tmp.positions[x]/10),file=f)
                i+=1

                ####new methyl H positions
                C5=tmp.select_atoms('name C5').positions[0]
                H5=tmp.select_atoms('name H5').positions[0]

                r=H5-C5

                new_H1=np.array([C5[0],C5[1]+r[1]*2,C5[2]+r[2]*2])
                new_H2=np.array([C5[0]+r[0]*2,C5[1],C5[2]+r[2]*2])
                new_H3=np.array([C5[0]+r[0]*2,C5[1]+r[1]*2,C5[2]])

                print(fmt.format(resid=tmp.resids[0],
                                resname="DT",
                                index=i,
                                name="H51",
                                pos=new_H1/10),file=f)
                i+=1

                print(fmt.format(resid=tmp.resids[0],
                                resname="DT",
                                index=i,
                                name="H52",
                                pos=new_H2/10),file=f)
                i+=1
                print(fmt.format(resid=tmp.resids[0],
                                resname="DT",
                                index=i,
                                name="H53",
                                pos=new_H3/10),file=f)
                i+=1
    else:
        for x in range(len(tmp)):
            if tmp.atoms.names[x]!="O2'" and tmp.atoms.names[x]!="H2'":
                print(fmt.format(     resid=tmp.resids[0],
                                    resname=tmp.resnames[0],
                                    index=i,
                                    name=tmp.atoms.names[x],
                                    pos=tmp.positions[x]/10),file=f)
                i+=1
            elif tmp.atoms.names[x]=="H2'":
                print(fmt.format(     resid=tmp.resids[0],
                                    resname=tmp.resnames[0],
                                    index=i,
                                    name=tmp.atoms.names[x],
                                    pos=tmp.positions[x-1]/10),file=f)
                i+=1


## Edit outputted gro file (temp_final.gro) so that you input a blank line followed by a line with the number of atoms (which can be found in the third column on the bottom line of the file).  And also add a line to the end of the file which is 10.0 10.0 10.0 ..

In [27]:
u3=MDAnalysis.Universe('temp_final.gro')
u3.atoms.write(f'{output_file}')